In [1]:
import cv as cv2
import numpy as np
import os

os.getcwd()

'C:\\Users\\bsterling\\Desktop\\3DBioImaging'

In [2]:
img_array = np.load('abs_mask.npy')

In [3]:
# pc = []
# scale = 2
# for n in range(len(img_array)):
#   contours, _ = cv2.findContours(img_array[n], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#   for i in range(len(contours[0])):
#     pc.append([contours[0][i][0][0],contours[0][i][0][1], n/scale])
# pc = np.array(pc)

In [4]:
pc = []
scale = 2
for n in range(len(img_array)):
    tmp = np.where(img_array[n] > 0)
    for i in range(len(tmp[0])):
        pc.append([tmp[0][i],tmp[1][i], n/scale])
pc = np.array(pc)

In [5]:
pc.shape

(203483, 3)

In [41]:
import pyvista as pv
import pyvistaqt as pvqt

In [42]:
# Note: ipyvtklink is needed (can be installed normally with pip)
cloud = pv.PolyData(pc)
cloud.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [43]:
volume = cloud.delaunay_3d(alpha=10.)

In [44]:
shell = volume.extract_geometry()

In [45]:
shell.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [11]:
p = pvqt.BackgroundPlotter()
p.set_background('white')
# p.show_bounds(grid='front', location='outer', all_edges=True)
p.add_bounding_box(line_width=2, color= 'black')
p.add_mesh(shell,opacity=0.7, smooth_shading=True,
                 silhouette=dict(line_width=1, color='grey'))
p.show()

In [12]:
volume = shell.volume
volume

13659.476713314885

In [13]:
bodies = shell.split_bodies()
# Now remove all bodies with a small volume
for key in bodies.keys():
    b = bodies[key]
    vol = b.volume
    if vol < 1000.0:
        del bodies[key]
        continue
    # Now lets add a volume array to all blocks
    b.cell_arrays["TOTAL VOLUME"] = np.full(b.n_cells, vol)

In [14]:
for i, body in enumerate(bodies):
    print(f"Body {i:02d} volume: {body.volume:.3f}")

Body 00 volume: 150073.857
Body 01 volume: 96935.606
Body 02 volume: 32262.220
Body 03 volume: 1446.673


In [15]:
for i, body in enumerate(bodies):
    surf = body.extract_surface()
    print(f"Body {i:02d} volume: {surf.area:.3f}")

Body 00 volume: 130099.037
Body 01 volume: 120335.018
Body 02 volume: 75048.612
Body 03 volume: 1373.771


In [16]:
bodies.plot(scalars="TOTAL VOLUME", cmap="jet", show_grid=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [17]:
surf = shell.extract_surface()
surf.plot(show_scalar_bar=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [18]:
surf = bodies[2].extract_surface()

In [19]:
p = pvqt.BackgroundPlotter()
p.add_mesh(surf)
p.show()

In [20]:
surf.area

75048.61203782899

In [21]:
smooth = surf.smooth(n_iter=1000)
smooth.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [22]:
p = pvqt.BackgroundPlotter()
p.add_mesh(smooth)
p.show()

In [23]:
qual = shell.compute_cell_quality(quality_measure='scaled_jacobian')
qual

PolyData (0x1e3b7e1ca60)
  N Cells:	96108
  N Points:	203483
  X Bounds:	9.300e+01, 4.380e+02
  Y Bounds:	1.120e+02, 3.960e+02
  Z Bounds:	0.000e+00, 2.200e+01
  N Arrays:	2

In [24]:
qual.plot(scalars='CellQuality')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [25]:
p = pvqt.BackgroundPlotter()
p.add_mesh(qual)
p.show()

In [27]:
import open3d as o3d

In [28]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc)
pcd.estimate_normals()

In [29]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 1.5 * avg_dist

In [30]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

In [31]:
dec_mesh = bpa_mesh.simplify_quadric_decimation(50)

In [32]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

TriangleMesh with 188310 points and 49 triangles.

In [33]:
o3d.visualization.draw_geometries([bpa_mesh])

In [34]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, width=0, scale=1.1, linear_fit=True)[0]

In [39]:
o3d.visualization.draw_geometries([pcd])

In [36]:
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

In [38]:
o3d.visualization.draw_geometries([p_mesh_crop])